# Lpoint MLB SKU 별 Sourcing

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()

db = os.getenv("CON_STR")

engine = create_engine(db)

## Total CC Purchaser 조회

In [2]:
# PMK CC, Comp CC >> Taste 추가
#  - user_past_type_M1 

sql3 = ''' 
with CC_Purchaser as ( 
	select
		t.YYYYMM, 
		t.id, 
	    CASE 
	        WHEN SUM(CASE WHEN b.cigatype = 'CC' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	         AND SUM(CASE WHEN b.cigatype = 'HnB' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	        THEN 'Mixed' 
	        ELSE MAX(b.cigatype)  -- CC 또는 HnB가 없을 경우 가장 큰 값을 사용
	    END AS cigatype
    from cx.seven11_user_3month_list t
		join cx.fct_K7_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM
		join cx.product_master b on a.product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'
	where t.YYYYMM >= '202408'
	group BY t.YYYYMM, t.id
	having sum(pack_qty) > 1
)
, temp as (
	select  
		t.YYYYMM, 
		t.id,
		b.ProductFamilyCode,
        b.FLAVORSEG_type3,
		b.New_TARSEGMENTAT,
		b.THICKSEG,
		sum(a.pack_qty) pack_qty,
		max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) 'PMK CC',
		max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) 'Comp CC',
		max(cast( case when b.cigatype='HnB' then 'HnB' else '' end as varchar(50) )) 'HnB'
	from CC_Purchaser t
		join cx.fct_K7_Monthly a on t.id = a.id and a.YYYYMM = t.YYYYMM	
		join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	where 1=1 
	group BY 	    
		t.YYYYMM, 
		t.id,
		b.ProductFamilyCode,
        b.FLAVORSEG_type3,
		b.New_TARSEGMENTAT,
		b.THICKSEG
)
select 
	YYYYMM,  
	id,
	New_TARSEGMENTAT,
    FLAVORSEG_type3,
	THICKSEG,
	pack_qty,
	usage_type
from temp
unpivot (usage_type for company in ([PMK CC], [Comp CC], [HnB]) ) as unpivo
where usage_type != ''
;
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

# past_type = data3.groupby(['id', 'YYYYMM'])['usage_type'].apply(lambda x: '+'.join(x)).reset_index()

user_past_Type = data3.groupby(['YYYYMM', 'usage_type', 'FLAVORSEG_type3']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(user_past_Type)
user_past_Type.to_clipboard()

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n (10060) (SQLGetData); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10060)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [3]:
user_past_Type.to_clipboard()

In [4]:
tar_type = data3.groupby(['YYYYMM',  'New_TARSEGMENTAT']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(tar_type)
tar_type.to_clipboard()

                            id  pack_qty
YYYYMM New_TARSEGMENTAT                 
202408                     907    3165.0
       1MG               79713  385880.4
       Below 1MG          9814   44611.0
       FF                 3015   12553.0
       LTS               42525  204396.1
       ULT               36084  158833.0
202409                    1014    3745.0
       1MG               78672  367156.3
       Below 1MG          9535   42470.0
       FF                 2954   11869.0
       LTS               41389  192760.1
       ULT               34923  149608.0


In [ ]:
thickness_type = data3.groupby(['YYYYMM',  'THICKSEG']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(thickness_type)
thickness_type.to_clipboard()

In [25]:
company_type = data3.groupby(['YYYYMM',  'usage_type']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(company_type)
company_type.to_clipboard()

                    id  pack_qty
YYYYMM usage_type               
202401 Comp CC     337    4483.9
       PMK CC      238    2592.0
202402 Comp CC     951    9555.8
       PMK CC      691    5211.0
202403 Comp CC     624    6164.1
       PMK CC      402    2566.0
202404 Comp CC     363    2918.9
       PMK CC      235    1212.0


## MLB Sourcing 작업 Tar, Taste, Thickness

In [15]:
#  PMK CC, Comp CC >> Tar Seg 추가

sql3 = ''' 
with temp as (
	select  
		t.YYYYMM, 
		t.id,
		b.engname,
		b.New_TARSEGMENTAT,
        b.FLAVORSEG_type3,
		b.THICKSEG,
		sum(a.pack_qty) pack_qty,
		max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) 'PMK CC',
		max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) 'Comp CC'
	from cx.agg_LPoint_MLB_SKU_Sourcing t
		join cx.fct_K7_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
		join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	where t.engname = 'Marlboro Vista Blossom Mist'
	group BY 	    
		t.YYYYMM, 
		t.id,
		b.engname,
		b.New_TARSEGMENTAT,
        b.FLAVORSEG_type3,
		b.THICKSEG
)
select 
	YYYYMM,  
	id,
	engname, 
	New_TARSEGMENTAT,
    FLAVORSEG_type3,
	THICKSEG,
	pack_qty,
	usage_type
from temp
unpivot (usage_type for company in ([PMK CC], [Comp CC]) ) as unpivo
where usage_type != ''
;
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

# past_type = data3.groupby(['id', 'YYYYMM'])['usage_type'].apply(lambda x: '+'.join(x)).reset_index()

user_past_Type1 = data3.groupby(['YYYYMM', 'New_TARSEGMENTAT']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

user_past_Type2 = data3.groupby(['YYYYMM', 'THICKSEG']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

user_past_Type3 = data3.groupby(['YYYYMM', 'usage_type', 'FLAVORSEG_type3']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(user_past_Type1)


                          id  pack_qty
YYYYMM New_TARSEGMENTAT               
202401 1MG               310    3664.4
       Below 1MG          72     387.0
       FF                 16     217.0
       LTS               158    1178.5
       ULT               197    1629.0
202402 1MG               941    8532.7
       Below 1MG         146     608.0
       FF                 46     186.0
       LTS               343    2401.1
       ULT               500    3039.0
202403 1MG               615    4997.0
       Below 1MG          65     252.0
       FF                 25     117.0
       LTS               233    1398.1
       ULT               288    1966.0
202404 1MG               351    2124.7
       Below 1MG          51     188.0
       FF                 26      85.0
       LTS               138     799.2
       ULT               175     934.0


In [19]:
user_past_Type3.to_clipboard()

In [24]:
# --  PMK CC 과거 구매내역 집계

sql3 = ''' 
with temp as (
	select  
		t.YYYYMM, 
		t.id,
		b.engname,
		b.ProductFamilyCode,
		b.New_TARSEGMENTAT,
		b.THICKSEG,
		sum(a.pack_qty) pack_qty,
		max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) 'PMK CC',
		max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) 'Comp CC'
	from cx.agg_LPoint_MLB_SKU_Sourcing t
		join cx.fct_K7_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
		join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	where 1=1 
	group BY 	    
		t.YYYYMM, 
		t.id,
		b.engname,
		b.ProductFamilyCode,
		b.New_TARSEGMENTAT,
		b.THICKSEG
)
select 
	YYYYMM,  
	id,
	engname, 
	New_TARSEGMENTAT,
	THICKSEG,
	pack_qty,
	usage_type
from temp 
unpivot (usage_type for company in ([PMK CC], [Comp CC]) ) as unpivo
where usage_type = 'PMK CC' and ProductFamilyCode = 'MLB'  --and engname like '%Vista%'
group by YYYYMM, ProductFamilyCode
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

# past_type = data3.groupby(['id', 'YYYYMM'])['usage_type'].apply(lambda x: '+'.join(x)).reset_index()

pm_cc_usage_Type = data3.groupby(['YYYYMM', 'engname']).agg({
    'id': 'nunique',
    'pack_qty': 'sum'
})

print(pm_cc_usage_Type)
pm_cc_usage_Type.to_clipboard()


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'unpivo.id' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW)")
[SQL:  
with temp as (
	select  
		t.YYYYMM, 
		t.id,
		b.engname,
		b.ProductFamilyCode,
		b.New_TARSEGMENTAT,
		b.THICKSEG,
		sum(a.pack_qty) pack_qty,
		max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) 'PMK CC',
		max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) 'Comp CC'
	from cx.agg_LPoint_MLB_SKU_Sourcing t
		join cx.fct_K7_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
		join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' A

# MLB 3개월 과거이력 조회

In [10]:
# - user_past_type_M1 CC MLB 용

sql3 = ''' 
with temp as (
select  
	t.YYYYMM, 
	t.id,
	t.engname,
	max(cast( case when b.cigatype='HnB' and b.company = 'PMK' then 'PMK HnB'   else '' end as varchar(50) )) A,
	max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) B,
	max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) C,
	max(cast( case when b.cigatype='HnB' and b.company != 'PMK' then 'Comp HnB' else '' end as varchar(50) )) D
from cx.agg_LPoint_MLB_SKU_Sourcing t
	join cx.fct_K7_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
where 1=1 --t.engname ='Marlboro Vista Blossom Mist'
group BY 	    
	t.YYYYMM, 
	t.id,
	t.engname
)
select 
	YYYYMM,  
	id,
	engname, 
	usage_type
from temp
unpivot (usage_type for company in (A, B, C, D) ) as unpivo
where usage_type != ''
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

past_type = data3.groupby(['id', 'YYYYMM','engname'])['usage_type'].apply(lambda x: '+'.join(x)).reset_index()

user_past_Type = past_type.groupby(['YYYYMM', 'engname', 'usage_type']).count()

print(user_past_Type)
user_past_Type.to_clipboard()


                                                                      id
YYYYMM engname                      usage_type                          
202401 Marlboro Vista Blossom Mist  Comp CC                          129
                                    Comp CC+Comp HnB                  29
                                    Comp HnB                           9
                                    PMK CC                            63
                                    PMK CC+Comp CC                   103
...                                                                  ...
202409 Marlboro Vista Garden Splash PMK HnB+Comp HnB                   4
                                    PMK HnB+PMK CC                    20
                                    PMK HnB+PMK CC+Comp CC            34
                                    PMK HnB+PMK CC+Comp CC+Comp HnB   13
                                    PMK HnB+PMK CC+Comp HnB            3

[85 rows x 1 columns]


In [12]:
# - user_Current_type_M1 CC MLB 용

sql4 = ''' 
with temp as (
select  
	t.YYYYMM, 
	t.id,
	t.engname,
	max(cast( case when b.cigatype='HnB' and b.company = 'PMK' then 'PMK HnB'   else '' end as varchar(50) )) A,
	max(cast( case when b.cigatype='CC'  and b.company = 'PMK' then 'PMK CC'    else '' end as varchar(50) )) B,
	max(cast( case when b.cigatype='CC'  and b.company != 'PMK' then 'Comp CC'  else '' end as varchar(50) )) C,
	max(cast( case when b.cigatype='HnB' and b.company != 'PMK' then 'Comp HnB' else '' end as varchar(50) )) D
from cx.agg_LPoint_MLB_SKU_Sourcing t
	join cx.fct_K7_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cx.product_master b on a.Product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
where 1=1 --t.engname ='Marlboro Vista Blossom Mist'
group BY 	    
	t.YYYYMM, 
	t.id,
	t.engname
)
select 
	YYYYMM,  
	id,
	engname, 
	usage_type
from temp
unpivot (usage_type for company in (A, B, C, D) ) as unpivo
where usage_type != ''
'''

data3 = pd.read_sql_query(sql=sql4, con=engine)

current_Type = data3.groupby(['id', 'YYYYMM','engname'])['usage_type'].apply(lambda x: '+'.join(x)).reset_index()

user_current_Type = current_Type.groupby(['YYYYMM', 'engname', 'usage_type']).count()

print(user_current_Type)
user_current_Type.to_clipboard()


                                                                      id
YYYYMM engname                      usage_type                          
202401 Marlboro Vista Blossom Mist  PMK CC                           119
                                    PMK CC+Comp CC                   213
                                    PMK CC+Comp CC+Comp HnB           43
                                    PMK CC+Comp HnB                   17
                                    PMK HnB+PMK CC                     9
                                    PMK HnB+PMK CC+Comp CC            20
                                    PMK HnB+PMK CC+Comp CC+Comp HnB    6
202402 Marlboro Vista Blossom Mist  PMK CC                           493
                                    PMK CC+Comp CC                   523
                                    PMK CC+Comp CC+Comp HnB           91
                                    PMK CC+Comp HnB                   74
                                    PMK HnB+PMK CC 

## MLB flavorXtar from 202401 
## user_past_type_M1, user_current_type_M1

In [ ]:
# - MLB flaXtar_ from 202401
sql2 = ''' 
select  
	t.YYYYMM,
	t.engname,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cx.agg_LPoint_MLB_SKU_Sourcing t
	join cx.fct_K7_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cx.product_master b on a.Product_code = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
where 1=1 
group BY 
	t.YYYYMM,
	t.engname,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
;
'''

data2 = pd.read_sql_query(sql=sql2, con=engine)

pivot_flavor_tar = data2.pivot_table(index=['YYYYMM','engname'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    ).sort_values(['engname', 'YYYYMM'])
print(pivot_flavor_tar)



In [13]:

# ALL Raw Data
sql1= ''' 
select * from cx.agg_LPoint_MLB_SKU_Sourcing
order by YYYYMM, id
'''

gg = pd.read_sql_query(sql=sql1, con=engine)

gg.reset_index(drop=1)

print(gg )
gg.to_clipboard()

      YYYYMM                                                 id  \
0     202401  012013BC75D985D37AC0263EE8A0EB240F93D25154EA23...   
1     202401  01A0FCE055ED96A6A3BA0EDD71800490723AB1C116CD48...   
2     202401  021FCAE970EA3D62BF46E9D7BD76B0ECCE1CB25F915938...   
3     202401  02A13EB4DF91D2688F48F38709DAD65E3C82A5F040ECCC...   
4     202401  02C9332ACC487585C8CF2503EE1DF9A36B2A7DF1C277C5...   
...      ...                                                ...   
4926  202409  FE5F682AFD371F8103A01FBB0B97D4779860CA71384824...   
4927  202409  FF18B1D4D3AE82544CEC4AAAF08AF9AEEBECFA73B7A9AF...   
4928  202409  FF29999988A549CBC96CBDCF281ED17E14D5DA8BDE9FEA...   
4929  202409  FF5DBB54007C453D4A083F89D8CC1FB3A835CBF0B33942...   
4930  202409  FFBD15AB4CCB7A9A31446CB9C626DA760A84AB1724603C...   

                           engname gender  age cigatype      company  \
0      Marlboro Vista Blossom Mist      남  20대    Mixed      JTI,KTG   
1      Marlboro Vista Blossom Mist      여   미상     